<a href="https://colab.research.google.com/github/ThatCodeCodingGuy/Text-Summarization-with-T-5-Pegasus-and-Bart-Transformers/blob/main/dataset_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openpyxl

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
df=pd.read_excel(r'/content/Inshorts Cleaned Data.xlsx')
df.head()

,Headline,Short,Source,Time,Publish Date
0,4 ex-bank officials booked for cheating bank o...,The CBI on Saturday booked four former officia...,The New Indian Express,09:25:00,2017-03-26
1,Supreme Court to go paperless in 6 months: CJI,Chief Justice JS Khehar has said the Supreme C...,Outlook,22:18:00,2017-03-25
2,"At least 3 killed, 30 injured in blast in Sylh...","At least three people were killed, including a...",Hindustan Times,23:39:00,2017-03-25
3,Why has Reliance been barred from trading in f...,Mukesh Ambani-led Reliance Industries (RIL) wa...,Livemint,23:08:00,2017-03-25
4,Was stopped from entering my own studio at Tim...,TV news anchor Arnab Goswami has said he was t...,YouTube,23:24:00,2017-03-25


In [6]:
df= df[['Headline','Short']]
df.head()

,Headline,Short
0,4 ex-bank officials booked for cheating bank o...,The CBI on Saturday booked four former officia...
1,Supreme Court to go paperless in 6 months: CJI,Chief Justice JS Khehar has said the Supreme C...
2,"At least 3 killed, 30 injured in blast in Sylh...","At least three people were killed, including a..."
3,Why has Reliance been barred from trading in f...,Mukesh Ambani-led Reliance Industries (RIL) wa...
4,Was stopped from entering my own studio at Tim...,TV news anchor Arnab Goswami has said he was t...


In [7]:
# Removing some common meaningless expressions
df=df.replace(r'&#45','-', regex=True).replace(r'&#39','', regex=True).replace(r'&amp','and', regex=True).replace(r'&#34;','', regex=True)
df.head()

,Headline,Short
0,4 ex-bank officials booked for cheating bank o...,The CBI on Saturday booked four former officia...
1,Supreme Court to go paperless in 6 months: CJI,Chief Justice JS Khehar has said the Supreme C...
2,"At least 3 killed, 30 injured in blast in Sylh...","At least three people were killed, including a..."
3,Why has Reliance been barred from trading in f...,Mukesh Ambani-led Reliance Industries (RIL) wa...
4,Was stopped from entering my own studio at Tim...,TV news anchor Arnab Goswami has said he was t...


In [8]:
# For quicker result, random 20 samples were chosen, this number can be tweaked
df=df.sample(n=20)
df.reset_index(inplace = True,drop = True)
df.head()

,Headline,Short
0,"Over 1,000 vehicles stranded in Assam, Tripura","According to reports, over 1,000 trucks filled..."
1,NGT asks AAP govt to discuss Delhi air quality,The National Green Tribunal on Tuesday asked t...
2,Campaign launched to name Android N ;Neyyappam;,Following Google;s announcement that it will c...
3,"No alliance for UP, Cong will go alone: Rahul",Congress Vice-President Rahul Gandhi has affir...
4,₹62 cr illegal cash seized in poll-bound states,The Election Commission has till Sunday seized...


In [9]:
df.isnull().sum()

Headline    0
Short       0
dtype: int64

In [10]:
''' I provided the full codes for using T-5, Pegasus and Bart transformers.
    Any of the code snippets below could be activated based on the transformer to be used ''' 
    
!pip install transformers sentencepiece

     |████████████████████████████████| 3.4 MB 5.7 MB/s 
     |████████████████████████████████| 1.2 MB 26.6 MB/s 
     |████████████████████████████████| 895 kB 46.2 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
#########################################
# T-5 in TensorFlow
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf",device=0)

#########################################

# Pegasus in TensorFlow
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
#from transformers import pipeline
#tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
#model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")
#summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf",device=0)


#########################################

# BART in PyTorch
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
#from transformers import pipeline
#tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
#model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
#summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, min_length=5, max_length=30, device=0)


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [12]:
l = []
for i in range(len(df['Short'])):
    l.append(summarizer(df['Short'][i], min_length=10, max_length=30))
df.loc[:,'News'] = l
df.head()

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1077: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


,Headline,Short,News
0,"Over 1,000 vehicles stranded in Assam, Tripura","According to reports, over 1,000 trucks filled...",{'summary_text': 'the repairing work is going ...
1,NGT asks AAP govt to discuss Delhi air quality,The National Green Tribunal on Tuesday asked t...,{'summary_text': 'the national green tribunal ...
2,Campaign launched to name Android N ;Neyyappam;,Following Google;s announcement that it will c...,{'summary_text': 'three Bengaluru-based techie...
3,"No alliance for UP, Cong will go alone: Rahul",Congress Vice-President Rahul Gandhi has affir...,{'summary_text': 'the grand-old party will not...
4,₹62 cr illegal cash seized in poll-bound states,The Election Commission has till Sunday seized...,{'summary_text': 'election commission has seiz...


In [13]:
df['News']=df['News'].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  20 non-null     object
 1   Short     20 non-null     object
 2   News      20 non-null     string
dtypes: object(2), string(1)
memory usage: 608.0+ bytes


In [14]:
df['News']=df.News.str[18:]
df['News']=df.News.str[:-2]
df.head()

,Headline,Short,News
0,"Over 1,000 vehicles stranded in Assam, Tripura","According to reports, over 1,000 trucks filled...",the repairing work is going on very slow for o...
1,NGT asks AAP govt to discuss Delhi air quality,The National Green Tribunal on Tuesday asked t...,the national green tribunal asks the governmen...
2,Campaign launched to name Android N ;Neyyappam;,Following Google;s announcement that it will c...,three Bengaluru-based techies have launched a ...
3,"No alliance for UP, Cong will go alone: Rahul",Congress Vice-President Rahul Gandhi has affir...,the grand-old party will not forge any allianc...
4,₹62 cr illegal cash seized in poll-bound states,The Election Commission has till Sunday seized...,election commission has seized cash from five ...
